### 1. The with Statement
The most common way to use context managers.

In [ ]:
# Without context manager - manual cleanup
file = open('example.txt', 'w')
try:
    file.write('Hello, World!')
finally:
    file.close()

# With context manager - automatic cleanup
with open('example.txt', 'w') as file:
    file.write('Hello, World!')
# File is automatically closed here

# Verify file is closed
print(f"File closed: {file.closed}")

# Read the file
with open('example.txt', 'r') as file:
    content = file.read()
    print(f"Content: {content}")

### 2. Creating Context Managers with Classes
Implement __enter__() and __exit__() methods.

In [ ]:
class FileManager:
    """Simple file manager context manager"""
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        """Called when entering the with block"""
        print(f"Opening file: {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, traceback):
        """Called when exiting the with block"""
        print(f"Closing file: {self.filename}")
        if self.file:
            self.file.close()
        
        # Return False to propagate exceptions
        # Return True to suppress exceptions
        return False

# Using custom context manager
with FileManager('test.txt', 'w') as f:
    f.write('Testing custom context manager')
    print("Writing to file...")

print("\nFile operations complete")

# Read back the content
with FileManager('test.txt', 'r') as f:
    print(f"Content: {f.read()}")

### 3. Exception Handling in Context Managers
Understanding __exit__() parameters.

In [ ]:
class DatabaseConnection:
    """Simulated database connection"""
    def __init__(self, connection_string):
        self.connection_string = connection_string
    
    def __enter__(self):
        print(f"Connecting to: {self.connection_string}")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is not None:
            print(f"\nException occurred: {exc_type.__name__}")
            print(f"Exception message: {exc_value}")
        
        print("Closing database connection")
        
        # Return True to suppress exception, False to propagate
        return False
    
    def query(self, sql):
        print(f"Executing: {sql}")
        return ["result1", "result2"]

# Normal execution
print("Normal execution:")
with DatabaseConnection("localhost:5432") as db:
    results = db.query("SELECT * FROM users")
    print(f"Results: {results}")

print("\n" + "="*50)

# With exception
print("\nExecution with exception:")
try:
    with DatabaseConnection("localhost:5432") as db:
        results = db.query("SELECT * FROM users")
        raise ValueError("Something went wrong!")
        print("This won't be printed")
except ValueError as e:
    print(f"\nCaught exception outside context manager: {e}")

### 4. Context Managers with contextlib
Using the @contextmanager decorator.

In [ ]:
from contextlib import contextmanager

@contextmanager
def file_manager(filename, mode):
    """Simple file manager using contextlib"""
    print(f"Opening {filename}")
    file = open(filename, mode)
    
    try:
        yield file  # This is where the with block executes
    finally:
        print(f"Closing {filename}")
        file.close()

# Using the context manager
with file_manager('test2.txt', 'w') as f:
    f.write('Hello from contextlib!')

with file_manager('test2.txt', 'r') as f:
    print(f"Content: {f.read()}")

### 5. Multiple Context Managers
Using multiple context managers in a single with statement.

In [ ]:
# Create source file
with open('source.txt', 'w') as f:
    f.write('Line 1\nLine 2\nLine 3\n')

# Multiple context managers
with open('source.txt', 'r') as source, open('destination.txt', 'w') as dest:
    content = source.read()
    dest.write(content.upper())
    print("File copied and transformed")

# Verify
with open('destination.txt', 'r') as f:
    print(f"Destination content:\n{f.read()}")

### 6. Practical Example: Timer Context Manager

In [ ]:
import time
from contextlib import contextmanager

@contextmanager
def timer(name="Operation"):
    """Time code execution"""
    start_time = time.time()
    print(f"Starting {name}...")
    
    try:
        yield
    finally:
        end_time = time.time()
        elapsed = end_time - start_time
        print(f"{name} took {elapsed:.4f} seconds")

# Using timer
with timer("Data processing"):
    # Simulate some work
    data = [i**2 for i in range(100000)]
    total = sum(data)
    print(f"Processed {len(data)} items, sum: {total}")

print("\n" + "="*50 + "\n")

with timer("Sleep operation"):
    time.sleep(1)

### 7. Directory Changer Context Manager

In [ ]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(path):
    """Temporarily change working directory"""
    original_dir = os.getcwd()
    print(f"Current directory: {original_dir}")
    
    try:
        os.chdir(path)
        print(f"Changed to: {os.getcwd()}")
        yield
    finally:
        os.chdir(original_dir)
        print(f"Restored to: {os.getcwd()}")

# Current directory
print(f"Working directory: {os.getcwd()}\n")

# Create a test directory
test_dir = 'test_directory'
os.makedirs(test_dir, exist_ok=True)

# Use context manager
with change_directory(test_dir):
    print(f"Inside context: {os.getcwd()}")
    # Create a file in the new directory
    with open('test_file.txt', 'w') as f:
        f.write('Test content')

print(f"\nAfter context: {os.getcwd()}")

### 8. Suppressing Exceptions

In [ ]:
from contextlib import suppress

# Without suppress
print("Without suppress:")
try:
    with open('nonexistent.txt', 'r') as f:
        content = f.read()
except FileNotFoundError:
    print("File not found - handled with try/except")

print("\nWith suppress:")
# With suppress - cleaner syntax
with suppress(FileNotFoundError):
    with open('nonexistent.txt', 'r') as f:
        content = f.read()
    print("This won't print if file doesn't exist")

print("Execution continues...")

# Suppress multiple exceptions
with suppress(FileNotFoundError, PermissionError, ValueError):
    # Any of these exceptions will be suppressed
    raise ValueError("This error is suppressed")

print("Still running after suppressed exception")

### 9. Redirect Standard Output

In [ ]:
from contextlib import redirect_stdout
import io

# Capture stdout
print("Normal print to console")

# Redirect to string buffer
f = io.StringIO()
with redirect_stdout(f):
    print("This goes to the buffer")
    print("So does this")
    for i in range(3):
        print(f"Number: {i}")

# Get captured output
captured = f.getvalue()
print("\nBack to normal console")
print(f"\nCaptured output:\n{captured}")

# Redirect to file
with open('output.txt', 'w') as f:
    with redirect_stdout(f):
        print("This goes to the file")
        print("Multiple lines")

print("\nReading from file:")
with open('output.txt', 'r') as f:
    print(f.read())

### 10. Transaction Context Manager

In [ ]:
class Transaction:
    """Simulate database transaction"""
    def __init__(self, name):
        self.name = name
        self.committed = False
    
    def __enter__(self):
        print(f"Beginning transaction: {self.name}")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:
            self.commit()
        else:
            self.rollback()
            print(f"Error: {exc_value}")
        return False
    
    def commit(self):
        print(f"Committing transaction: {self.name}")
        self.committed = True
    
    def rollback(self):
        print(f"Rolling back transaction: {self.name}")
        self.committed = False

# Successful transaction
print("Successful transaction:")
with Transaction("User Registration") as txn:
    print("  - Creating user account")
    print("  - Sending welcome email")
    print("  - Logging activity")

print(f"Transaction committed: {txn.committed}")

print("\n" + "="*50 + "\n")

# Failed transaction
print("Failed transaction:")
try:
    with Transaction("Payment Processing") as txn:
        print("  - Validating card")
        print("  - Processing payment")
        raise ValueError("Insufficient funds")
        print("  - This won't execute")
except ValueError:
    print(f"Transaction committed: {txn.committed}")

### 11. Lock Context Manager for Threading

In [ ]:
import threading
import time

# Shared resource
counter = 0
lock = threading.Lock()

def increment_counter(name, times):
    global counter
    for _ in range(times):
        # Using lock as context manager
        with lock:
            current = counter
            time.sleep(0.001)  # Simulate some work
            counter = current + 1
    print(f"{name} finished")

# Without lock, there would be race conditions
threads = []
for i in range(3):
    thread = threading.Thread(target=increment_counter, args=(f"Thread-{i+1}", 10))
    threads.append(thread)
    thread.start()

# Wait for all threads
for thread in threads:
    thread.join()

print(f"\nFinal counter value: {counter}")
print(f"Expected value: {3 * 10}")

### 12. Nested Context Managers

In [ ]:
from contextlib import contextmanager, ExitStack

@contextmanager
def section(name):
    print(f"\n{'='*40}")
    print(f"Entering: {name}")
    print(f"{'='*40}")
    try:
        yield
    finally:
        print(f"\nExiting: {name}")
        print(f"{'='*40}")

# Nested context managers
with section("Outer Section"):
    print("Doing outer work...")
    
    with section("Inner Section"):
        print("Doing inner work...")
        print("More inner work...")
    
    print("Back to outer work...")

print("\nAll done!")

### 13. Dynamic Context Managers with ExitStack

In [ ]:
from contextlib import ExitStack

# Create multiple files dynamically
filenames = ['file1.txt', 'file2.txt', 'file3.txt']

# Using ExitStack to manage multiple files
with ExitStack() as stack:
    # Open all files and add to stack
    files = [stack.enter_context(open(fname, 'w')) for fname in filenames]
    
    # Write to all files
    for i, f in enumerate(files):
        f.write(f"Content for file {i+1}\n")
        print(f"Written to {f.name}")
    
    # All files automatically closed when exiting the with block

# Verify files were created and closed
print("\nReading files:")
for fname in filenames:
    with open(fname, 'r') as f:
        print(f"{fname}: {f.read().strip()}")